# Step 02: Control Totals

This notebook executes control totals SQL scripts to generate validation data for the extracted CSV files.

**Tasks:**
- Load configuration metadata (date_value, cycle_type)
- Execute 3a_Control_Totals_Working_Table.sql
- Execute 3b_Control_Totals_Contract_Import_File_Tables.sql
- Display results for review

## 1) Setup

In [ ]:
%load_ext autoreload
%autoreload 2

from helpers.notebook_setup import initialize_notebook_context
from helpers import ux
from helpers.database import execute_query
from helpers.sqlserver import execute_query_from_file, sql_file_exists
from helpers.constants import WORKSPACE_PATH

# Flag to track execution state
execution_failed = False
error_message = None

In [ ]:
# Initialize notebook context and step tracking
context, step = initialize_notebook_context('Step_02_Control_Totals.ipynb')

# Display context
ux.header("Control Totals Execution")
ux.info(f"Cycle: {context.cycle_name}")
ux.info(f"Stage: {context.stage_name}")
ux.info(f"Step: {context.step_name}")
ux.success(f"\u2713 Step tracking initialized for '{context.step_name}'")

## 2) Load Configuration

In [ ]:
# Load configuration to get date_value and cycle_type
ux.subheader("Load Configuration")

# Query for active configuration
query = """
    SELECT c.configuration_data
    FROM irp_configuration c
    INNER JOIN irp_cycle cy ON c.cycle_id = cy.id
    WHERE cy.cycle_name = %s
      AND c.status IN ('VALID', 'ACTIVE')
    ORDER BY c.created_ts DESC
    LIMIT 1
"""

result = execute_query(query, (context.cycle_name,))

if result.empty:
    execution_failed = True
    error_message = "No valid configuration found for this cycle"
    ux.error(f"\u2717 {error_message}")
else:
    config_data = result.iloc[0]['configuration_data']
    metadata = config_data.get('Metadata', {})
    
    date_value = metadata.get('Current Date Value', '')
    cycle_type = metadata.get('Cycle Type', '')
    
    if not date_value or not cycle_type:
        execution_failed = True
        error_message = f"Missing required metadata: date_value={date_value}, cycle_type={cycle_type}"
        ux.error(f"\u2717 {error_message}")
    else:
        ux.success(f"\u2713 Configuration loaded")
        ux.info(f"  Date Value: {date_value}")
        ux.info(f"  Cycle Type: {cycle_type}")
        step.log(f"Configuration loaded: date_value={date_value}, cycle_type={cycle_type}")

## 3) Execute Working Table Control Totals (3a)

In [ ]:
# Execute 3a_Control_Totals_Working_Table.sql
working_table_results = []

if execution_failed:
    ux.warning("\u23ed Skipping 3a execution due to configuration error")
else:
    ux.subheader("Execute Working Table Control Totals")
    
    sql_file_3a = WORKSPACE_PATH / 'sql' / 'control_totals' / '3a_Control_Totals_Working_Table.sql'
    
    if not sql_file_exists(sql_file_3a):
        ux.warning(f"\u26a0 SQL file not found: {sql_file_3a}")
        ux.info("Skipping 3a execution")
    else:
        ux.info(f"Executing: {sql_file_3a.name}")
        ux.info(f"Parameters: DATE_VALUE={date_value}")
        ux.info("")
        
        try:
            # Execute SQL script - returns list of DataFrames (one per SELECT)
            working_table_results = execute_query_from_file(
                sql_file_3a,
                params={'DATE_VALUE': date_value},
                connection='ASSURANT',
                database='DW_EXP_MGMT_USER'
            )
            
            ux.success(f"\u2713 Executed 3a script: {len(working_table_results)} result set(s)")
            step.log(f"Executed 3a script: {len(working_table_results)} result sets")
            
        except Exception as e:
            ux.error(f"\u2717 Error executing 3a script: {str(e)}")
            step.log(f"Error executing 3a script: {str(e)}")

In [ ]:
# Display 3a results
if working_table_results:
    ux.subheader("Working Table Control Totals (3a Results)")
    
    # Define section labels based on SQL script comments
    section_labels = [
        "CB EQ",
        "CB HU",
        "US EQ",
        "US FF",
        "US ST",
        "US HU Leak",
        "US HU Full",
        "US WF",
        "US FL Commercial Flood",
        "US FL Excess Flood",
        "US FL Other Flood"
    ]
    
    for i, df in enumerate(working_table_results):
        label = section_labels[i] if i < len(section_labels) else f"Result Set {i+1}"
        ux.info(f"\n--- {label} ---")
        
        if df.empty:
            ux.info("  (No data)")
        else:
            # Convert DataFrame to list of lists for ux.table
            headers = df.columns.tolist()
            rows = df.values.tolist()
            
            # Format numeric values
            formatted_rows = []
            for row in rows:
                formatted_row = []
                for val in row:
                    if isinstance(val, (int, float)) and not isinstance(val, bool):
                        formatted_row.append(f"{val:,.0f}" if val == int(val) else f"{val:,.2f}")
                    else:
                        formatted_row.append(str(val) if val is not None else '')
                formatted_rows.append(formatted_row)
            
            ux.table(formatted_rows, headers=headers)
else:
    if not execution_failed:
        ux.info("No 3a results to display")

## 4) Execute Import File Control Totals (3b)

In [ ]:
# Execute 3b_Control_Totals_Contract_Import_File_Tables.sql
import_file_results = []

if execution_failed:
    ux.warning("\u23ed Skipping 3b execution due to configuration error")
else:
    ux.subheader("Execute Import File Control Totals")
    
    sql_file_3b = WORKSPACE_PATH / 'sql' / 'control_totals' / '3b_Control_Totals_Contract_Import_File_Tables.sql'
    
    if not sql_file_exists(sql_file_3b):
        ux.warning(f"\u26a0 SQL file not found: {sql_file_3b}")
        ux.info("Skipping 3b execution")
    else:
        ux.info(f"Executing: {sql_file_3b.name}")
        ux.info(f"Parameters: DATE_VALUE={date_value}, CYCLE_TYPE={cycle_type}")
        ux.info("")
        
        try:
            # Execute SQL script - returns list of DataFrames (one per SELECT)
            import_file_results = execute_query_from_file(
                sql_file_3b,
                params={'DATE_VALUE': date_value, 'CYCLE_TYPE': cycle_type},
                connection='ASSURANT',
                database='DW_EXP_MGMT_USER'
            )
            
            ux.success(f"\u2713 Executed 3b script: {len(import_file_results)} result set(s)")
            step.log(f"Executed 3b script: {len(import_file_results)} result sets")
            
        except Exception as e:
            ux.error(f"\u2717 Error executing 3b script: {str(e)}")
            step.log(f"Error executing 3b script: {str(e)}")

In [ ]:
# Display 3b results
if import_file_results:
    ux.subheader("Import File Control Totals (3b Results)")
    
    # Define section labels based on SQL script comments
    section_labels = [
        "CB EQ",
        "CB HU",
        "US EQ",
        "US FF",
        "US ST",
        "US HU Leak",
        "US HU Full",
        "US WF",
        "US FL Commercial",
        "US FL Excess",
        "US FL Other"
    ]
    
    for i, df in enumerate(import_file_results):
        label = section_labels[i] if i < len(section_labels) else f"Result Set {i+1}"
        ux.info(f"\n--- {label} ---")
        
        if df.empty:
            ux.info("  (No data)")
        else:
            # Convert DataFrame to list of lists for ux.table
            headers = df.columns.tolist()
            rows = df.values.tolist()
            
            # Format numeric values
            formatted_rows = []
            for row in rows:
                formatted_row = []
                for val in row:
                    if isinstance(val, (int, float)) and not isinstance(val, bool):
                        formatted_row.append(f"{val:,.0f}" if val == int(val) else f"{val:,.2f}")
                    else:
                        formatted_row.append(str(val) if val is not None else '')
                formatted_rows.append(formatted_row)
            
            ux.table(formatted_rows, headers=headers)
else:
    if not execution_failed:
        ux.info("No 3b results to display")

## 5) Complete Step Execution

In [ ]:
# Complete step execution
ux.header("Step Completion")

if execution_failed:
    # Handle configuration/execution failure
    from helpers.step import update_step_run
    from helpers.constants import StepStatus
    
    update_step_run(step.run_id, StepStatus.FAILED, error_message=error_message)
    
    ux.error("\n" + "="*60)
    ux.error("CONTROL TOTALS EXECUTION FAILED")
    ux.error("="*60)
    ux.error(f"\nError: {error_message}")
    ux.info("\nPlease fix the error and retry.")

else:
    # Complete the step successfully
    output_data = {
        'date_value': date_value,
        'cycle_type': cycle_type,
        'working_table_result_count': len(working_table_results),
        'import_file_result_count': len(import_file_results)
    }
    step.complete(output_data)

    ux.success("\n" + "="*60)
    ux.success("CONTROL TOTALS EXECUTED SUCCESSFULLY")
    ux.success("="*60)
    ux.info(f"\nWorking Table (3a): {len(working_table_results)} result set(s)")
    ux.info(f"Import File (3b): {len(import_file_results)} result set(s)")
    ux.info("\nReview the control totals above to validate the data extraction.")
    ux.info("\nNext: Proceed to Stage 03 Data Import")